In [1]:
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt
import seaborn as sns
from btd import BTD

In [2]:
data = np.load('../dat/methylation/breast_and_ovarian/data.npz')

In [3]:
Beta_IJ = data['Beta_IJ']             # IxJ matrix of beta-values
sample_labels = data['sample_labels'] # I-length list of labels for each sample
locus_labels = data['locus_labels']   # J-length list of labels for each locus

In [4]:
(I, J) = Beta_IJ.shape
assert len(sample_labels) == I
assert len(locus_labels) == J
print I, J

200 5000


In [5]:
C = 6          # number of sample clusters
K = 8          # number of locus clusters

b = 1.           # shape of background fluoresence noise
e = 0.1          # shape parameter for prior over factor matrices
f = 1.           # rate parameter for prior over factor matrices
eta = 0.1        # shape parameter for prior over core matrix
gam = 30         # prior rate of CpG dinucleotides in each locus
seed = 111111    # random seed (optional)
dirichlet = True # use Dirichlet priors (use them)

model = BTD(I=I, J=J, C=C, K=K, b=b, e=e, f=f, eta=eta, gam=gam, dirichlet=dirichlet, seed=seed)

In [6]:
num_itns = 500            # number of Gibbs sampling iterations to run
verbose = True            # whether to printout every iteration
initialize = True         # whether to randomly initialize parameters; initialization runs for 40 iterations

burnin = {'c_I': np.inf}  # dictionary where keys are parameter names and values are the... 
                          # ...earliest iteration when to start sampling them.  If set to...
                          # ...infinity, the param wont be fit.  Don't sample c_I.

model.fit(Beta_IJ, num_itns=num_itns, verbose=verbose, burnin=burnin, initialize=initialize)


INITIALIZING...

46.6571ms: sampling Lambda_2IJ
191.4799ms: sampling Y_2IJ
46.8001ms: sampling shp_2CK
0.0010ms: sampling shp_IC
0.0010ms: sampling shp_JK
0.1230ms: sampling Theta_IC
3.6180ms: sampling Phi_JK
0.0122ms: sampling Pi_CK
0.0110ms: sampling c_I
3 95317
ITERATION 1: sparsity: 0.046529, num_tokens: 95317

47.1561ms: sampling Lambda_2IJ
204.8161ms: sampling Y_2IJ
53.3061ms: sampling shp_2CK
0.0012ms: sampling shp_IC
0.0010ms: sampling shp_JK
0.1149ms: sampling Theta_IC
4.3259ms: sampling Phi_JK
0.0131ms: sampling Pi_CK
0.0100ms: sampling c_I
3 101344
ITERATION 2: sparsity: 0.049328, num_tokens: 101344

54.2538ms: sampling Lambda_2IJ
198.7250ms: sampling Y_2IJ
53.2031ms: sampling shp_2CK
0.0021ms: sampling shp_IC
0.0000ms: sampling shp_JK
0.1171ms: sampling Theta_IC
4.8001ms: sampling Phi_JK
0.0129ms: sampling Pi_CK
0.0100ms: sampling c_I
3 101961
ITERATION 3: sparsity: 0.049640, num_tokens: 101961

51.4178ms: sampling Lambda_2IJ
198.5321ms: sampling Y_2IJ
48.8429ms: sampling 

In [7]:
state = model.get_state()
Pi_CK = state['Pi_CK']        # CxK core matrix
Theta_IC = state['Theta_IC']  # IxC sample-cluster factor matrix
Phi_JK = state['Phi_JK']      # JxK loci-cluster factor matrix
Y_2IJ = state['Y_2IJ']        # 2xIxJ latent Poisson counts
                              # Y_2IJ[0] are the methylated counts
                              # Y_2IJ[1] are the unmethylated counts